In [ ]:
from google.colab import drive

In [ ]:
drive.mount("/content/gdrive/")

In [ ]:
import pandas as pd

In [ ]:
path="/content/gdrive/MyDrive/titanic (2)/"
train=pd.read_csv(path+"train.csv")
test=pd.read_csv(path+"test.csv")

In [ ]:
train["Survived"].value_counts(normalize=True) #생존자 비율

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
sns.countplot(train['Survived'])

In [ ]:
train["Pclass"]

In [ ]:
train['Survived'].groupby(train['Pclass']).mean()


In [ ]:
sns.countplot(train['Pclass'], hue=train['Survived'])


In [ ]:
train['Name'].head()


In [ ]:
#호칭별 인원수(정규식 없이)
train['Name_Title']=train['Name'].apply(lambda x:x.split(",")[1]).apply(lambda x:x.split()[0])
train['Name_Title']

In [ ]:
train['Name_Title'].value_counts()

In [ ]:
train['Survived'].groupby(train['Name_Title']).mean()


In [ ]:
train['Name_Len'] = train['Name'].apply(lambda x: len(x))


In [ ]:
train["Name_Len"]
#이름의 길이를 구간으로 나눈 다음, 구간별 생존율?
train['Survived'].groupby(pd.qcut(train['Name_Len'],5)).mean()

In [ ]:
train['Sex'].value_counts(normalize=True)

In [ ]:
train['Survived'].groupby(train['Sex']).mean()

In [ ]:
train['Survived'].groupby(train['Age'].isnull()).mean()

In [ ]:
train.info()

In [ ]:
train['Survived'].groupby(pd.qcut(train['Age'],5)).mean()


In [ ]:
pd.qcut(train['Age'],5).value_counts(normalize=True)


In [ ]:
train['Survived'].groupby(train['SibSp']).mean()


In [ ]:
train['SibSp'].value_counts()


In [ ]:
train['Survived'].groupby(train['Parch']).mean()


In [ ]:
train['Parch'].value_counts()


In [ ]:
train['Ticket'].head(n=10)


In [ ]:
#티켓 길이에 따른 생존율
train['Ticket_Len'] = train['Ticket'].apply(lambda x: len(x))


In [ ]:
train.groupby(['Ticket_Len'])['Survived'].mean()


In [ ]:
train['Ticket_Len'].value_counts()


In [ ]:
train["Ticket"]

In [ ]:
#티켓의 앞글자와 생존율티켓의 앞글자와 생존율
train['Ticket_Lett'] = train['Ticket'].apply(lambda x: str(x)[0])


In [ ]:
train['Ticket_Lett'].value_counts()


In [ ]:
train.groupby(['Ticket_Lett'])['Survived'].mean()


In [ ]:
#티켓 운임에 따른 생존율티켓 운임에 따른 생존율
pd.qcut(train['Fare'], 3).value_counts()



In [ ]:
train['Survived'].groupby(pd.qcut(train['Fare'], 3)).mean()


In [ ]:
#요금에 따른 객실등급표
pd.crosstab(pd.qcut(train['Fare'], 5), columns=train['Pclass'])


In [ ]:
#cabin 첫글자에 따른 생존율
train['Cabin_Letter'] = train['Cabin'].apply(lambda x: str(x)[0])


In [ ]:
train['Cabin_Letter'].value_counts()


In [ ]:
train['Survived'].groupby(train['Cabin_Letter']).mean()


In [ ]:
train['Cabin']

In [ ]:
#숫자 부분만 추출
train['Cabin_num']=train['Cabin'].apply(lambda x:str(x).split()[-1][1:])

In [ ]:
train['Cabin_num'].replace("an",np.NaN,inplace=True)

In [ ]:
train['Cabin_num']

In [ ]:
#문자로 되어있던 숫자를 숫자로 변경 NaN을 진짜 NaN으로 변경
train["Cabin_num"]=train["Cabin_num"].apply(lambda x:int(x) if not pd.isnull(x) and x !="" else np.NaN) 
# x가 null이 아니고 공백이 아니라면 int로 바꾸고 나머지는  NaN으로 바꾼다

In [ ]:
pd.qcut(train['Cabin_num'],3).value_counts()

In [ ]:
train['Survived'].groupby(pd.qcut(train['Cabin_num'], 3)).mean()


In [ ]:
train['Survived'].corr(train['Cabin_num'])
#거의 의미가 없는 수치임

In [ ]:
#Embarked와 생존율
train['Embarked'].value_counts(normalize=True)


In [ ]:
train['Survived'].groupby(train['Embarked']).mean()


In [ ]:
sns.countplot(train['Embarked'], hue=train['Pclass'])


In [ ]:
#feature engineering

In [ ]:
def name(train, test):
    for i in [train,test]:
        i["Name_Len"]=i["Name"].apply(lambda x:len(x))
        i['Name_Title'] = i['Name'].apply(lambda x: x.split(',')[1]).apply(lambda x: x.split()[0])
        del i['Name']

    return train,test

In [ ]:
train=pd.read_csv(path+"train.csv")
test=pd.read_csv(path+"test.csv")

In [ ]:
train,test=name(train,test)

In [ ]:
train

In [ ]:
def age_impute(train,test):
    for i in [train,test]:
        i['Age_Null_Flag'] = i['Age'].apply(lambda x: 1 if pd.isnull(x) else 0) # age에 uull값은 1 아니면 0
    train['mean'] = train.groupby(['Name_Title', 'Pclass'])['Age'].transform('mean') 
    train['Age'] = train['Age'].fillna(train['mean'])
    z = test.merge(train, on=['Name_Title', 'Pclass'], how='left').drop_duplicates(['PassengerId_x'])
    test['Age'] = np.where(test['Age'].isnull(), z['mean'], test['Age'])
    test['Age'] = test['Age'].fillna(test['Age'].mean())
    del train['mean']



    return train,test

In [ ]:
train,test=age_impute(train,test)

In [ ]:
train.info()
test.info()

In [ ]:
def fam_size(train, test):
    for i in [train, test]:
        i['Fam_Size'] = np.where((i['SibSp']+i['Parch']) == 0 , 'Solo',
                           np.where((i['SibSp']+i['Parch']) <= 3,'Nuclear', 'Big'))
        del i['SibSp']
        del i['Parch']
    return train, test


In [ ]:
train, test=fam_size(train, test)

In [ ]:
def ticket_grouped(train, test):
    for i in [train, test]:
        i['Ticket_Len'] = i['Ticket'].apply(lambda x: len(x))
        del i['Ticket']
    return train, test


In [ ]:
train, test = ticket_grouped(train, test)


In [ ]:
def cabin_num(train, test):
    for i in [train, test]:
        i['Cabin_num1'] = i['Cabin'].apply(lambda x: str(x).split(' ')[-1][1:])
        i['Cabin_num1'].replace('an', np.NaN, inplace = True)
        i['Cabin_num1'] = i['Cabin_num1'].apply(lambda x: int(x) if not pd.isnull(x) and x != '' else np.NaN)
        i['Cabin_num'] = pd.qcut(train['Cabin_num1'],3)
    train = pd.concat((train, pd.get_dummies(train['Cabin_num'], prefix = 'Cabin_num')), axis = 1)
    test = pd.concat((test, pd.get_dummies(test['Cabin_num'], prefix = 'Cabin_num')), axis = 1)
    del train['Cabin_num']
    del test['Cabin_num']
    del train['Cabin_num1']
    del test['Cabin_num1']
    return train, test


In [ ]:
train, test=cabin_num(train, test)

In [ ]:
def cabin(train, test):
    for i in [train, test]:
        i['Cabin_Letter'] = i['Cabin'].apply(lambda x: str(x)[0])
        del i['Cabin']
    return train, test


In [ ]:
train, test=cabin(train, test)

In [ ]:
def embarked_impute(train, test):
    for i in [train, test]:
        i['Embarked'] = i['Embarked'].fillna('S')
    return train, test


In [ ]:
train, test = embarked_impute(train, test)


In [ ]:
def dummies(train, test, columns = ['Pclass', 'Sex', 'Embarked', 'Ticket_Lett', 'Cabin_Letter', 'Name_Title', 'Fam_Size']):
    for column in columns:
        train[column] = train[column].apply(lambda x: str(x))
        test[column] = test[column].apply(lambda x: str(x))
        good_cols = [column+'_'+i for i in train[column].unique() if i in test[column].unique()]
        train = pd.concat((train, pd.get_dummies(train[column], prefix = column)[good_cols]), axis = 1)
        test = pd.concat((test, pd.get_dummies(test[column], prefix = column)[good_cols]), axis = 1)
        del train[column]
        del test[column]
    return train, test


In [ ]:
train, test = dummies(train, test, columns = ['Pclass', 'Sex', 'Embarked', 
                                              'Cabin_Letter', 'Name_Title', 'Fam_Size'])


In [ ]:
train.head()

In [ ]:
def drop(train, test, bye = ['PassengerId']):
    for i in [train, test]:
        for z in bye:
            del i[z]
    return train, test


In [ ]:
train, test = drop(train, test)


In [ ]:
train

In [ ]:
test.info()
test["Fare"].fillna(train["Fare"].mean(), inplace=True)

In [ ]:
#하이퍼 퍼러미터 튜닝

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
rf=RandomForestClassifier(max_features="auto",
                       random_state=1,
                       oob_score=True,#out-of-bag: 트레이닝 과정에 사용되지 않은 데이터들로 평가를 하겠다.
                       n_jobs=-1)# 컴퓨터의 모든 cpu를 동원하겠다
                                

In [ ]:
## 안에 속성을 요소를 모두 짝지어서 다 모델을 만들어줌 아래에 경우 15번 모델을 만듬
param_grid={
    "n_estimators":[50,100,400,700,1000],
    "max_depth":[5,7,9]
}

In [ ]:
gs=GridSearchCV(estimator=rf,
             param_grid=param_grid,
             n_jobs=-1,
             cv=3)#cv:교차 검증시 분할되는 폴드의 수

In [ ]:
gs.fit(train.iloc[:,1:],train.iloc[:,0])

In [ ]:
print(gs.best_params_) 
print(gs.best_score_)

In [ ]:
## 모델을 만든 결과 최고는ㄴ
# 'max_depth': 9, 'n_estimators': 100}
# print(gs.best_score_) 위 값에 모델에 점수

In [ ]:
#위에서 알아낸 최고의 결과가 나오는 옵션을 넣어서 모델 작성
rf=RandomForestClassifier(max_features="auto",
                       random_state=1,
                       oob_score=True,#out-of-bag: 트레이닝 과정에 사용되지 않은 데이터들로 평가를 하겠다.
                       n_jobs=-1,
                       max_depth=9,
                       n_estimators=100)

In [ ]:
rf.fit(train.iloc[:,1:],train.iloc[:,0])

In [ ]:
predictions=rf.predict(test)

In [ ]:
predictions=pd.DataFrame(predictions,columns=["Survived"])

In [ ]:
test=pd.read_csv(path+"test.csv")

In [ ]:
test
predictions=pd.concat((test.iloc[:,0],predictions),axis=1)

In [ ]:
predictions.to_csv("myprediction.csv",index=False)

In [ ]:
##더미화 원핫 인코딩 get_dummies함수

In [ ]:
music_df = pd.DataFrame({'music_id': [1, 2, 3, 4, 5], 
                      'music_genre': ['rock', 
                                      'disco', 
                                      'pop', 
                                      'rock', 
                                      'pop']}
                      , columns = ['music_id', 'music_genre'])


In [ ]:
music_dummy_mat = pd.get_dummies(music_df['music_genre'])
music_dummy_mat 

In [ ]:
music_dummy_mat = music_df.join(music_dummy_mat.add_prefix('genre_'))
music_dummy_mat

In [ ]:
# 그룹별 연산
# transfor함수:스칼라 값 or 같은 크기를 갖는 배열이 리턴
# apply함수 :스칼라 값이 리턴

In [ ]:
tips=sns.load_dataset("tips")
tips

In [ ]:
tips.groupby("day").apply(np.mean)

In [ ]:
tips.groupby("day").transform(np.mean) #np.mean의 결과가 각 요소마다 치환됨 244행이라면 244행이 평균결과로 바뀜

In [ ]:
z=test.merge(train,on=["Name_Title","Pclass"],how="left").drop_duplicates(["PassengerId_x"])
z

In [ ]:
test['Age'] = np.where(test['Age'].isnull(), z['mean'], test['Age'])
